In [144]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import pandas as pd
import matplotlib.pyplot as plt
import time
import random
import requests
import numpy as np

In [145]:
import platform
from matplotlib import font_manager, rc

# 코드 테스트

In [5]:
driver_path = 'C:/Users/User/Desktop/chromedriver/chromedriver_win32/chromedriver'
driver = webdriver.Chrome(driver_path)

url = 'https://music.bugs.co.kr/chart/track/realtime/total'
driver.get(url)
time.sleep(2)

C:\Users\User\AppData\Local\Temp/ipykernel_1540/1055193298.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path)


## frame 추출

In [6]:
soup = bs(driver.page_source,'lxml')

In [7]:
raw_table = soup.find('table', 'list trackList byChart')
raw_table2 = raw_table.find('tbody')
len(raw_table2)

201

In [8]:
len(raw_table2.find_all('th'))

100

## 제목 리스트 만들기

In [9]:
song_name_list = []

for area in raw_table2.find_all('th'):
    name_area = area.find('p').a['title']
    song_name_text = name_area.strip()
    song_name_list.append(song_name_text)

print(len(song_name_list))
print(song_name_list[:5])

100
['봄여름가을겨울 (Still Life)', 'LOVE DIVE', 'TOMBOY', 'Feel My Rhythm', 'GANADARA (Feat. 아이유)']


## 아티스트, 앨범 리스트 만들기

In [10]:
len(raw_table2.find_all('td', 'left'))

200

In [11]:
count = 0

for area in raw_table2.find_all('td', 'left'):
    count +=1
    
    print(area.text)
    
    if count > 5: break



BIGBANG (빅뱅)



봄여름가을겨울 (Still Life)



IVE (아이브)



LOVE DIVE



(여자)아이들



I NEVER DIE



In [12]:
artist_album_list = []

for area in raw_table2.find_all('td', 'left'):
    artist_album_list.append(area.text.strip())
    
artist_album_list[:10]

['BIGBANG (빅뱅)',
 '봄여름가을겨울 (Still Life)',
 'IVE (아이브)',
 'LOVE DIVE',
 '(여자)아이들',
 'I NEVER DIE',
 'Red Velvet (레드벨벳)',
 '‘The ReVe Festival 2022 - Feel My Rhythm’',
 '박재범',
 'GANADARA']

td 태그의 class=left는 2개가 있습니다. 여기에는 가수 이름과 앨범이 포함되는데, 이들을 분리하여 각각 리스트를 만들어줍니다.

In [13]:
artist_list = artist_album_list[::2]
album_list = artist_album_list[1::2]

In [14]:
print(len(artist_list), len(album_list))

100 100


## 데이터프레임 만들기

In [15]:
rank = list(range(1, 101))

In [16]:
df = pd.DataFrame({'순위': rank,
                   '곡': song_name_list,
                   '아티스트': artist_list,
                   '앨범': album_list
                  })

df.shape

(100, 4)

In [17]:
df.head()

,순위,곡,아티스트,앨범
0,1,봄여름가을겨울 (Still Life),BIGBANG (빅뱅),봄여름가을겨울 (Still Life)
1,2,LOVE DIVE,IVE (아이브),LOVE DIVE
2,3,TOMBOY,(여자)아이들,I NEVER DIE
3,4,Feel My Rhythm,Red Velvet (레드벨벳),‘The ReVe Festival 2022 - Feel My Rhythm’
4,5,GANADARA (Feat. 아이유),박재범,GANADARA


In [18]:
df.isnull().sum()

순위      0
곡       0
아티스트    0
앨범      0
dtype: int64

# 벅스뮤직 차트 크롤링 함수 만들기

In [146]:
def bugs_df(url):
    driver_path = 'C:/Users/User/Desktop/chromedriver/chromedriver_win32/chromedriver'
    driver = webdriver.Chrome(driver_path)

    driver.get(url)
    time.sleep(2)
    
    soup = bs(driver.page_source,'lxml')

    raw_table = soup.find('table', 'list trackList byChart')
    raw_table2 = raw_table.find('tbody')
    
    # 제목 리스트 만들기
    song_name_list = []

    for area in raw_table2.find_all('th'):
        name_area = area.find('p').a['title']
        song_name_text = name_area.strip()
        song_name_list.append(song_name_text)
        
    # 아티스트, 앨범 리스트 만들기
    artist_album_list = []

    for area in raw_table2.find_all('td', 'left'):
        artist_album_list.append(area.text.strip())

    artist_list = artist_album_list[::2]
    album_list = artist_album_list[1::2]
    
    # 데이터프레임 만들기
    rank = list(range(1, 101))
    
    df = pd.DataFrame({'순위': rank,
                   '곡': song_name_list,
                   '아티스트': artist_list,
                   '앨범': album_list
                  })
    
    df.head()
    
    random_number = random.uniform(1, 2)
    time.sleep(random_number)
    
    return df

## bugs_df에서 나온 df 저장하는 함수

In [20]:
from tqdm import tqdm

from datetime import datetime
from dateutil.relativedelta import relativedelta

In [160]:
def save_df(start_date, end_date):
    # 원하는 날짜 리스트에 담기
    # pd.date_range에 원하는 날짜 입력하기
    
    while True:
    
        start_date = str(start_date)

        # 20220103-> 2022, 01, 03
        start_year = start_date[:4]
        start_month = start_date[4:6]
        start_day = start_date[6:]
        
        date1 = start_year + '-' + start_month + '-' + start_day

        url = "https://music.bugs.co.kr/chart/track/week/total?chartdate="
        url_with_date = url + start_date
        
        # df추출 후 저장
        df = bugs_df(url_with_date)

        # DataFrame.inser(loc,column, value, allow_duplicates=False)
        df.insert(0, '날짜', date1)
        # 파일 저장
        save_path = f"C:/Users/User/Desktop/음원사재기/벅스/week_bugs_{start_date}.xlsx"
        df.to_excel(save_path, index=False)
        
        # start_date에 + 1 week 해준 이후 다시 end_date와 비교할 수 있게 변환
        plus_year=int(start_year)
        if start_month == '10':
            plus_month = int(start_month)
        else:
            plus_month = int(start_month.replace('0', ''))
        
        ten_day_list = ['10','20', '30']
        if start_day in ten_day_list:
            plus_day = int(start_day)
        else:
            plus_day = int(start_day.replace('0', ''))
        
        plus_element = datetime(plus_year, plus_month, plus_day) + relativedelta(days=7)
        temp_date = str(plus_element)[:10]
        start_date = int(temp_date.replace('-', ''))
        
        if start_date > end_date:
            break
        
        df

In [163]:
# yyyymmdd 형식으로 input 넣어야 함

save_df(20220214, 20220331)

C:\Users\User\AppData\Local\Temp/ipykernel_1540/308752351.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path)
